<a href="https://colab.research.google.com/github/chrober24/SL_2021-2022/blob/main/stylegan2_tutorial_train_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## stylegan2: Training

this notebook contains scripts for training a stylegan2 model as well as various scripts for generating images from a trained network and projecting into the network

Training requires connecting colab to google drive as the training process generates a large amount of data that cannot be hosted locally on colab.

# setup

In [ ]:
#@title Set tensorflow version
#@markdown always use tensorflow1

%tensorflow_version 1.x


In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install StyleGAN2 Repository
#@markdown StyleGAN2 will be installed to your Google Drive to speed up the training process

#@markdown Run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.
import os
import shlex
if os.path.isdir("/content/drive/MyDrive/stylegan2-aug-colab"):
  %cd "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2"
elif os.path.isdir("/content/drive"):
  %cd /content/drive/MyDrive/
  !mkdir stylegan2-aug-colab
  %cd stylegan2-aug-colab/
  !git clone -b augs-attn https://github.com/dvschultz/stylegan2
  %cd stylegan2
  !mkdir pkl
  %cd pkl
  !gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
  %cd ../
  !mkdir results
  !mkdir results/00001-pretrained
  %cd results/00001-pretrained
  !gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
  %cd ../../
  %mkdir datasets
else:
  !git clone -b augs-attn https://github.com/dvschultz/stylegan2
  %cd styelgan2
  !mkdir pkl
  %cd pkl
  !gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
  %cd ../
  !mkdir results
  !mkdir results/00001-pretrained
  %cd results/00001-pretrained
  !gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
  %cd ../../
  %mkdir datasets


# Create tfrecord 
(only run at beginning of training)


*   first path: output dataset - only change last part of the folder path to your dataset name. (/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/{your-dataset-name-here})
*  Second path: folder of prepared images - must be square (512 max) 24bit pngs. (use [Dataset-Prep](https://github.com/chrober24/SL_2021-2022/blob/main/Dataset_Prep.ipynb) notebook to prepare images)




In [5]:
!python dataset_tool.py create_from_images_raw /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/tex /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/raw_images/tex

Loading images from "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/raw_images/tex"
detected 251 images ...
Shuffle the images...
Creating dataset "/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/datasets/tex"
Adding the images to tfrecords ...
added images 0
Added 251 images.


# Pre-train model download
**run only once!**

Run this cell if you need a pretrained model to perform transfer learning.
this will download Nvidia's model trained on cars.

Additional pretrained models can be found [here](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/)
and [here](https://github.com/justinpinkney/awesome-pretrained-stylegan2)


In [11]:
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-car-config-f.pkl -o /content/drive/MyDrive/stylegan2-aug-colab/stylegan2/pkl/stylegan2-car-config-f.pkl

# Training

In the next cell set your pkl and `resume_kimg` counter. If this is your first time the settings below should work for you.



*   **Transfer Learning:** 
  * `pkl` should be set to a pretrained model
  * `resume_kimg` should be set to 10000
*   **resuming a previous Training session**
  * `pkl` should be set to the path of your most recent checkpoint (found in results folder)
  * `resume_kimg` should match the number in the filename of the pkl.
  
**Replace `--dataset` with your dataset name** (set in the tfrecord step above)




In [ ]:
#@title Transfer/Resuming Options
pkl="/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/pkl/stylegan2-car-config-f.pkl" #@param
resume_kimg = 10000 #@param
dataset = "dataset_name" #@param

In [ ]:
#@title TRAIN
#@markdown the `AUG_PROB` environment setting tells the training loop how often to modify the real and fake images with augmentations. The default is `0.1` If you have a small training set you may want to go higher than that but note that the Karras paper does say if you set this value too high you may find it bleeds into the outputs.

!AUG_PROB=0.2 python run_training.py --num-gpus=1 --mirror-augment=True --data-dir=./datasets --dataset=$dataset --config=config-f --resume-pkl=$pkl --resume-kimg=$resume_kimg --augmentations=True --metrics=None